In [3]:
import os
import sys

import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt


import nltk
import random
import re

# Download NLTK resources (if not downloaded)
nltk.download('punkt')

from utils import *

In [28]:
# generate csv data file for each label
dir_data = 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data'
dir_human = 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\heste-nettet-nyheder'
dir_ai4 = 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\heste-nettet-nyheder-ai\\gpt-4-0613'
dir_ai3 = 'C:\\Users\\Simon Andersen\\Projects\\Projects\\DMIAI_2023\\aiText\\data\\heste-nettet-nyheder-ai\\gpt-3.5-turbo'


# find all subdirectories and subsubdirectories
dir_paths = [os.path.join(dir_data, f) for f in os.listdir(dir_data) if os.path.isdir(os.path.join(dir_data, f))][:-1]

# add subdirectories not already in dir_paths
dirs_to_pop = []
for dir_path in dir_paths:
    sub_dir_paths = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, f))]

    if len(sub_dir_paths) > 0:
        dir_paths.extend(sub_dir_paths) 
        dirs_to_pop.append(dir_path)

for dir_to_pop in dirs_to_pop:
    dir_paths.remove(dir_to_pop)

dirs_human = dir_paths[:2]
dirs_ai = dir_paths[2:]


In [36]:
# find all txt files in the directory
paths_human = []
paths_ai = []
for dir_hum, dir_ai in zip(dirs_human,dirs_ai):
    paths_human.extend(find_txt_files(dir_hum))
    paths_ai.extend(find_txt_files(dir_ai))

Nai = len(paths_ai)
Nhuman = len(paths_human)

paths_human = paths_human[:Nai]
Nhuman = len(paths_human)   
print(f"Number of human files: {Nhuman}")
print(f"Number of ai files: {Nai}")

Number of human files: 1380
Number of ai files: 1380


In [33]:
text_human = read_human_files(paths_human)
text_ai = read_files(paths_ai, line_start = 3)


In [37]:
# create a dataframe with the text and label (human = 0 or ai = 1) for each text list
df_human = pd.DataFrame({'text': text_human, 'label': [0]*len(text_human)})
df_ai = pd.DataFrame({'text': text_ai, 'label': [1]*len(text_ai)})

df= pd.concat([df_human, df_ai], ignore_index=True)

# shuffle the dataframe
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# save the dataframe as a csv file
df.to_csv('data.csv', index=False)

In [46]:
# Approximate the sentence length distribution of the validation set
def len_dist(mean, std):
    positive = False
    while not positive:
        length = int(norm(mean, std).rvs())
        if length > 1:
            positive = True
    return length

In [47]:
# Provide the path to your text file
text_file_path = ['brevkasse.txt', 'news2.txt', 'public_information.txt']
base_path = os.path.splitext(text_file_path)[0]

In [47]:
# Provide the path to text files
text_file_paths = ['brevkasse.txt', 'news2.txt', 'public_information.txt']
# Construct the generator
rvs_generator = lambda: len_dist(4, 3)

for text_file_path in text_file_paths:
    base_path = base_path = os.path.splitext(text_file_path)[0]

    # Get the list of strings
    resulting_strings = split_text_into_strings(text_file_path, rvs_generator)
    df_str = pd.DataFrame({'text': resulting_strings})
    df_str.to_csv(f'{base_path}.csv', index=False)

    print(f"Number of strings: {len(resulting_strings)}")

    if 0:
        # Print the resulting strings
        for i, string in enumerate(resulting_strings, 1):
            print(f"String {i}:")
            print(string)
            print("\n" + "=" * 50 + "\n")

Number of strings: 193
Number of strings: 214
Number of strings: 96
